In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from utils import display_image, display_images
if cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

##### 1. Apply custom sharpening kernel of aperture size 3 and 5 as shown below on 'native-bee.png':  
$ 3 \times 3$ kernel:  
$ \begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0\\
\end{bmatrix}$  
$ 5 \times 5$ kernel:  
$ \begin{bmatrix}
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & 25 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
\end{bmatrix}$  
What can you infer from the outputs?

In [2]:
img = cv.imread("images/native-bee.png")

kernel1 = np.array([[0, -1, 0],
                  [-1, 5, -1],
                  [0, -1, 0]])
kernel2 = np.ones((5,5)) * -1
kernel2[2][2] = 25
dst1 = cv.filter2D(img, -1, kernel1)
dst2 = cv.filter2D(img, -1, kernel2)

display_images([img, dst1, dst2], ("original", "sharpen(3X3)","sharpen(5X5)"))

kernel1(3x3) is a smaller  sharpening kernel, which provides a moderate sharpening effect.
kernel2(5x5) kernel with a stronger central value (25), will produce a overly sharpening effect.

##### 2. Apply different image smoothing techniques (e.g. average filter, Gaussian kernel and median filter) on 'noise_lena.jpg' and display the resulting images after the convolution. Comment on the outcomes and deduce the type of noise present on the image.

In [3]:
img = cv.imread("images/noise_lena.jpg")

#Average Filter
avg_filter = cv.blur(img, (5,5))
#Gaussian Kernel
gaussian_filter = cv.GaussianBlur(img, (5,5), 0)
#Median Filter
median_filter = cv.medianBlur(img, 5)
display_images([img, avg_filter, gaussian_filter, median_filter], 
               ("Original", "Average Filter", "Gaussian Filter", "Median Filter"))


Average Filter
The image show slight blurry but still contains noise, meaning that the average filter is not completely remove the noise. The noise may be salt and pepper noise. Therefore, the average filter are not perform well since it used for averaging but not eliminating noise.

Gaussian Filter
Gaussian Filter perform badly in this noised image. Gaussion filter are specifically designed to reduce gaussion noise, so indicate that the noise is likely of a different type.

Median Filter
The median filter performs exceptionally well, removing all noise without significant degradation of the image. This strongly indicates that the primary noise type in the image is salt-and-pepper noise since median filter is particularly effective against this type of noise.

##### 3. Write a program to *segment the boat and the people on it from the background*. Follow the instruction below:
    - Use 'boat.jpg' as input.
    - Apply Otsu thresholding.
    - Draw bounding box to identify the region where the boat and people are located.

In [17]:
img = cv.imread("images/boat.jpg")
display_image("color",img)

In [28]:
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
th = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
contours, _ = cv.findContours(th, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contour_max_area = sorted(contours, key = cv.contourArea, reverse = True)[0]
x,y,w,h = cv.boundingRect(contour_max_area)
img_copy = img.copy()
cv.rectangle(img_copy,(x,y),(x+w,y+h),(0,255,0),2)
display_image("BoundingBox",img_copy)